Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Extract Zip File

In [ ]:
import zipfile
import os

# ZIP file path
zip_path = '/content/drive/MyDrive/archive (2).zip'
extract_dir = '/content'

if os.path.exists(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f'Successfully extracted: {zip_path}')
    except zipfile.BadZipFile:
        print('Error: Not a valid zip file.')
    except Exception as e:
        print(f'An error occurred: {e}')
else:
    print('Error: ZIP file does not exist.')


Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


Set Data Paths and Image Size

In [ ]:
train_path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
img_size = 224


Train Data Generator

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
).flow_from_directory(
    train_path,
    batch_size=164,
    target_size=(img_size, img_size),
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True
)


Validation Data Generator

In [ ]:
valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(
    valid_path,
    batch_size=164,
    target_size=(img_size, img_size),
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)


Visualize Training Images

In [ ]:
classes = list(train_generator.class_indices.keys())
plt.figure(figsize=(20, 20))

for x_batch, y_batch in train_generator:
    n = min(16, len(x_batch))
    for i in range(n):
        plt.subplot(4, 4, i + 1)
        plt.imshow(x_batch[i])
        plt.title(classes[np.argmax(y_batch[i])])
        plt.axis('off')
    plt.show()
    break


Build CNN Model

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 7, padding="same", activation="relu", input_shape=(224, 224, 3), name="Conv1"),
    keras.layers.MaxPooling2D(pool_size=2, name="Pool1"),

    keras.layers.Conv2D(64, 5, padding="same", activation="relu", name="Conv2"),
    keras.layers.MaxPooling2D(pool_size=2, name="Pool2"),

    keras.layers.Conv2D(128, 3, padding="same", activation="relu", name="Conv3"),
    keras.layers.MaxPooling2D(pool_size=2, name="Pool3"),

    keras.layers.Conv2D(256, 3, padding="same", activation="relu", name="Conv4"),

    keras.layers.Flatten(name="Flatten"),
    keras.layers.Dense(128, activation="relu", name="Dense1"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu", name="Dense2"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(38, activation="softmax", name="Output")
])

model.summary()


Compile the Model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy', 'precision', 'recall']
)


Set Callbacks

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_lr=1e-5)

callbacks = [early_stopping, model_checkpoint, reduce_lr]


 Train the Model

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 36/429 ━━━━━━━━━━━━━━━━━━━━ 4:43:58 43s/step - accuracy: 0.0305 - loss: 4.0970 - precision: 0.0325 - recall: 0.0017

Plot Training Metrics

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
precision = history.history['precision']
val_precision = history.history['val_precision']
recall = history.history['recall']
val_recall = history.history['val_recall']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure(figsize=(12, 6))
plt.plot(epochs, acc, 'g', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True)
plt.show()


Evaluate the Model

In [ ]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(
    valid_path,  # Assuming valid is used for testing
    batch_size=64,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)

model_evaluate = model.evaluate(test_generator)

print('Loss:', model_evaluate[0])
print('Accuracy:', model_evaluate[1])
print('Precision:', model_evaluate[2])
print('Recall:', model_evaluate[3])


Save Model and Class Indices

In [ ]:
model.save('PDDS.keras')

import json
with open('class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)
